In [ ]:
from classy import Class
import matplotlib.pyplot as plt
from scipy.interpolate import LSQUnivariateSpline
from scipy.special import eval_legendre
import numpy as np
import os

os.makedirs("xis", exist_ok=True)

In [ ]:
cosmo = Class()
cosmo.set({
    'omega_b':      0.02238,      # physical baryon density
    'omega_cdm':    0.11600,      # physical cold dark matter density
    'h':            0.68,         # reduced hubble parameter
    'A_s':          2.1e-9,       # primordial scalar amplitude
    'n_s':          0.97,         # scalar spectral index
    'tau_reio':     0.054,        # reionization optical depth
})
cosmo.set({'output':'tCl,pCl,lCl,nCl,mPk','lensing':'yes','P_k_max_1/Mpc': 3.0, 'z_max_pk': 100})
cosmo.compute()

In [ ]:
kk = np.logspace(-4,np.log10(3),1000) # k in h/Mpc
Pk = [] # P(k) in (Mpc/h)**3
h = 0.68 # for conversions to 1/Mpc
for k in kk:
    Pk.append(cosmo.pk(k*h,0.)*h**3) # why isnt this working

Pk = np.array(Pk)

In [ ]:
r = np.linspace(1, 160, 400)  # r in Mpc/h; domain chosen mostly arbitrarily

xi_rad = np.zeros_like(r)

for i, ri in enumerate(r):
    kr = kk * ri
    # sin(kr)/(kr) = sinc(kr/pi)
    
    window = 0.5 * ( 1 - np.tanh( 2 * (kk - kk[-1] + 1.5) ) ) # Lepori et. al: "...we introduced a cutoff W to smooth numerical spurious oscillations..."
    integrand = kk**2 * Pk * np.sinc(kr / np.pi) * window
    xi_rad[i] = np.trapz(integrand, kk)

xi_rad /= 2.0 * np.pi**2

In [ ]:
xi_trans = np.zeros_like(r)

for i, ri in enumerate(r):
    kr = kk * ri
    # sin(kr)/(kr) = sinc(kr/pi)
    
    window = 0.5 * ( 1 - np.tanh( 2 * (kk - kk[-1] + 1.5) ) ) # Lepori et. al: "...we introduced a cutoff W to smooth numerical spurious oscillations..."
    legendre = eval_legendre(i, kr)
    integrand = kk**2 * Pk * np.sinc(kr / np.pi) * window * legendre
    xi_trans[i] = np.trapz(integrand, kk)

xi_trans /= 2.0 * np.pi**2

In [ ]:
np.savetext("xi/xi_rad.txt", xi_rad)
np.savetext("xi/xi_trans.txt", xi_trans)